In [1]:
# make sure to reload imports
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
from collections import defaultdict, deque
import re
from typing import List, Set, Tuple, Dict, Optional, TypedDict

# ─── Add parent directory to sys.path for relative imports ─────────────────────
sys.path.append(os.path.abspath(".."))

# ─── Static‐analysis imports ───────────────────────────────────────────────────
from analyzers.repo_analyzer import RepoAnalyzer
from verifiers.hw1 import APEHW1
from analyzers.function_slicer import FunctionSlicer

# ─── 1. Initialize analyzer & slicer ───────────────────────────────────────────
REPO_PATH = '../tmp/adiprerepa/cs598APE-hw1'
pv       = APEHW1(REPO_PATH)
analyzer = RepoAnalyzer.from_path(REPO_PATH, pv)
FUNCTION_SIG = "calcColor(unsigned char *, Autonoma *, Ray, unsigned int)"
slicer: FunctionSlicer = analyzer.nodes[FUNCTION_SIG]['slicer']

✅ libclang loaded from: /usr/lib/llvm-18/lib/libclang.so
✅ Uber file generated at /home/ayuram/pesquared/tmp/adiprerepa/cs598APE-hw1/__uber.cpp
✅ Library subgraph loaded from /home/ayuram/.pesquared/cache/lib_subgraph.gpickle
Found 6267 nodes
Found 6606 edges
Found Makefile: /home/ayuram/pesquared/tmp/adiprerepa/cs598APE-hw1/Makefile


sudo: a terminal is required to read the password; either use the -S option to read from standard input or configure an askpass helper
sudo: a password is required


In [ ]:
# ─── LLM / pipeline imports ────────────────────────────────────────────────────
from langchain_text_splitters import MarkdownHeaderTextSplitter
from llm import UniversalLLM
from prompts import (
    ALGORITHM_OUTLINE_PROMPT,
    ALGORITHM_BOTTLENECK_PROMPT,
    ARCHITECTURE_BOTTLENECK_PROMPT,
    OPTIMIZATION_PROMPT
)
from langgraph.graph import END, StateGraph

# ─── 2. Top‑3 bottleneck picker (stub) ─────────────────────────────────────────
def get_top_bottleneck_lines(fn: str) -> List[int]:
    # TODO: replace with real analysis
    return [67, 58, 104]

# ─── 3. Define state & basic LLM nodes ─────────────────────────────────────────
class PerfState(TypedDict, total=False):
    function_name: str
    code: str
    bottlenecks: List[int]
    algorithm_steps: str
    algorithm_bottlenecks: str
    architecture_bottlenecks: str
    optimized_function: str
    algo_note_0: str
    algo_note_1: str
    algo_note_2: str
    arch_note_0: str
    arch_note_1: str
    arch_note_2: str

universal = UniversalLLM(model="Llama-3.3-8B-Instruct", provider="meta", temperature=0)

def analyze_algorithmic_bottlenecks(state: PerfState) -> PerfState:
    prompt = ALGORITHM_BOTTLENECK_PROMPT.format(
        function_name=state["function_name"],
        code=state["code"],
        steps=state.get("algorithm_steps","")
    )
    result = universal.prompt(prompt)
    splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[
        ("#",    "Header 1"),
        ("##",   "Header 2"),
        ("###",  "Header 3"),
        ("####", "Notes"),
    ])
    splits = splitter.split_text(result)
    notes = [
        s.page_content.strip()
        for s in splits
        if s.metadata.get("Notes", "").lower().startswith("note to self")
    ]
    return {"algorithm_bottlenecks": "\n- ".join(notes).strip()}

def analyze_architecture_bottlenecks(state: PerfState) -> PerfState:
    prompt = ARCHITECTURE_BOTTLENECK_PROMPT.format(
        function_name=state["function_name"],
        code=state["code"]
    )
    return {"architecture_bottlenecks": universal.prompt(prompt).strip()}

def optimize_function(state: PerfState) -> PerfState:
    prompt = OPTIMIZATION_PROMPT.format(
        function_name=state["function_name"],
        code=state["code"],
        algorithm_bottlenecks=state.get("algorithm_bottlenecks",""),
        architecture_bottlenecks=state.get("architecture_bottlenecks",""),
    )
    result = universal.prompt(prompt)
    splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[
        ("#",    "Header 1"),
        ("##",   "Header 2"),
        ("###",  "Header 3"),
        ("####", "Notes"),
    ])
    splits = splitter.split_text(result)
    for s in splits:
        if state["function_name"].lower() in s.metadata.get("Header 1","").lower():
            return {**state, "optimized_function": s.page_content.strip()}
    return {**state, "optimized_function": result.strip()}

# ─── 4. Generate bottlenecks list ──────────────────────────────────────────────
def gen_bottlenecks(state: PerfState) -> PerfState:
    return {"bottlenecks": get_top_bottleneck_lines(state["function_name"])}

# ─── 5. Process one bottleneck line (factory) ─────────────────────────────────
def make_process_fn(idx: int):
    def process_one(state: PerfState) -> PerfState:
        ln = state["bottlenecks"][idx]
        slice_code = slicer.slice_snippet(ln, order=1, collapse_threshold=5)
        sub: PerfState = {
            "function_name": f"{state['function_name']} [line {ln}]",
            "code": slice_code
        }
        sub.update(analyze_algorithmic_bottlenecks(sub))
        # ensure arch key always present
        if not sub.get("algorithm_bottlenecks"):
            sub.update(analyze_architecture_bottlenecks(sub))
        return {
            f"algo_note_{idx}": sub.get("algorithm_bottlenecks",""),
            f"arch_note_{idx}": sub.get("architecture_bottlenecks","")
        }
    return process_one

process_b0 = make_process_fn(0)
process_b1 = make_process_fn(1)
process_b2 = make_process_fn(2)

# ─── 6. Aggregate branch notes ─────────────────────────────────────────────────
def aggregate_notes(state: PerfState) -> PerfState:
    algo = [
        f"**Line {ln}:**\n{state.get(f'algo_note_{i}', '')}"
        for i, ln in enumerate(state["bottlenecks"])
    ]
    arch = [
        f"**Line {ln}:**\n{state.get(f'arch_note_{i}', '')}"
        for i, ln in enumerate(state["bottlenecks"])
    ]
    return {
        "algorithm_bottlenecks": "\n\n".join(algo).strip(),
        "architecture_bottlenecks": "\n\n".join(arch).strip()
    }

# ─── 7. Build the single LangGraph ────────────────────────────────────────────
graph = StateGraph(PerfState)
graph.set_entry_point("gen_bottlenecks")
graph.add_node("gen_bottlenecks", gen_bottlenecks)

for name in ("process_b0", "process_b1", "process_b2"):
    graph.add_node(name, globals()[name])
    graph.add_edge("gen_bottlenecks", name)

graph.add_node("aggregate", aggregate_notes)
for name in ("process_b0", "process_b1", "process_b2"):
    graph.add_edge(name, "aggregate")

graph.add_node("optimize", optimize_function)
graph.add_edge("aggregate", "optimize")
graph.add_edge("optimize", END)

compiled = graph.compile()

# ─── 8. Invoke on initial state and print result ─────────────────────────────
initial_state: PerfState = {
    "function_name": FUNCTION_SIG,
    "code":     analyzer.nodes[FUNCTION_SIG]["code"]
}
final = compiled.invoke(initial_state)

print("\n===== Final Optimized Function =====\n")
print(final["optimized_function"])


===== Final Optimized Function =====

```cpp
void calcColor(unsigned char* toFill, Autonoma* c, Ray ray, unsigned int depth) {
ShapeNode* t = c->listStart;
std::vector<TimeAndShape> times;
while (t != NULL) {
double time = t->data->getIntersection(ray);
times.push_back({ time, t->data });
t = t->next;
}
// Find the minimum time value and its associated shape
TimeAndShape minTime = times[0];
for (const auto& time : times) {
if (time.time < minTime.time) {
minTime = time;
}
}
double curTime = minTime.time;
Shape* curShape = minTime.shape;
Vector intersect = curTime * ray.vector + ray.point;
double opacity, reflection, ambient;
curShape->getColor(toFill, &ambient, &opacity, &reflection, c, Ray(intersect, ray.vector), depth);

double lightData[3];
getLight(lightData, c, intersect, curShape->getNormal(intersect), curShape->reversible());
toFill[0] = (unsigned char)(toFill[0] * (ambient + lightData[0] * (1 - ambient)));
toFill[1] = (unsigned char)(toFill[1] * (ambient + lightData[1] * (1 - 